# IMPORTANDO BIBLIOTECAS NECESSÁRIAS

In [1]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
import os
import json
import datetime
stemmer = LancasterStemmer()

In [2]:
# Base de dados de treinamento, 12 frases pertencentes a 3 classes ("intenções"). 

In [3]:
# 3 Classe de dados de treinamento
training_data = []
training_data.append({"class":"greeting", "sentence":"how are you?"})
training_data.append({"class":"greeting", "sentence":"how is your day?"})
training_data.append({"class":"greeting", "sentence":"good day"})
training_data.append({"class":"greeting", "sentence":"how is it going today?"})

training_data.append({"class":"goodbye", "sentence":"have a nice day"})
training_data.append({"class":"goodbye", "sentence":"see you later"})
training_data.append({"class":"goodbye", "sentence":"have a nice day"})
training_data.append({"class":"goodbye", "sentence":"talk to you soon"})

training_data.append({"class":"sandwich", "sentence":"make me a sandwich"})
training_data.append({"class":"sandwich", "sentence":"can you make a sandwich?"})
training_data.append({"class":"sandwich", "sentence":"having a sandwich today?"})
training_data.append({"class":"sandwich", "sentence":"what's for lunch?"})
print ("%s sentences in training data" % len(training_data))

12 sentences in training data


In [4]:
# Organizando nossas estruturas de dados para documentos, classes e palavras. 

In [5]:
words = []
classes = []
documents = []
ignore_words = ['?']
# percorrer cada frase em nosss dados de treinamento 
for pattern in training_data:
    # tokenizar cada palavra na frase 
    w = nltk.word_tokenize(pattern['sentence'])
    # adicionar para a lista de palavras
    words.extend(w)
    # dicionar aos documentos em nosso corpus 
    documents.append((w, pattern['class']))
    # adicionar a nossa lista de classes
    if pattern['class'] not in classes:
        classes.append(pattern['class'])

# radicalizar e diminuir cada palavra e remover duplicatas 
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = list(set(words))

# remover duplicadas
classes = list(set(classes))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

12 documents
3 classes ['greeting', 'sandwich', 'goodbye']
26 unique stemmed words ['can', 'nic', 'today', 'good', 'lat', 'is', 'you', 'to', 'for', 'day', 'lunch', 'me', 'what', 'talk', 'a', 'going', "'s", 'how', 'soon', 'hav', 'sandwich', 'it', 'ar', 'see', 'mak', 'yo']


In [6]:
# Criando os dados de treinamento
training = []
output = []
# Criando um array vazio para a saída
output_empty = [0] * len(classes)

# conjunto de treinamento, saco de palavras para cada frase 
for doc in documents:
    # Inicializando nosso saco de palavras
    bag = []
    # lista de palavras tokenizadas para o padrão 
    pattern_words = doc[0]
    # radical de cada palavra 
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # criando nossa coleção de palavras 
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    training.append(bag)
    # a saída é um '0' para cada tag e '1' para a tag atual 
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    output.append(output_row)

# amostra de treinamento / saída 
i = 0
w = documents[i][0]
print ([stemmer.stem(word.lower()) for word in w])
print (training[i])
print (output[i])

['how', 'ar', 'you', '?']
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0]
[1, 0, 0]


In [7]:
# Cada frase de treinamento é reduzida a uma matriz de 0 e 1 contra a matriz de palavras únicas no corpus. 

In [8]:
# Em seguida, temos nossas funções principais para a rede neural de 2 camadas.
# Usando numpy porque queremos que a multiplicação da nossa matriz seja rápida.

# Usaremos uma função sigmóide para normalizar os valores e sua derivada para medir a taxa de erro. Iterando e ajustando 
# até que nossa taxa de erro seja aceitavelmente baixa.
# Também abaixo, implementamos a função de saco de palavras, transformando uma frase de entrada em uma matriz de 0 e 1. 
# Isso corresponde precisamente à nossa transformação para dados de treinamento, sempre crucial para acertar. 

In [9]:
import numpy as np
import time

# computar não linearidade sigmóide 
def sigmoid(x):
    output = 1/(1+np.exp(-x))
    return output

# converter a saída da função sigmóide em sua derivada 
def sigmoid_output_to_derivative(output):
    return output*(1-output)
 
def clean_up_sentence(sentence):
    # tokenizar o padrão
    sentence_words = nltk.word_tokenize(sentence)
    # radical de cada palavra
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# retorno do array de saco de palavras: 0 ou 1 para cada palavra na bolsa que existe na frase 
def bow(sentence, words, show_details=False):
    # tokenizar o padrão
    sentence_words = clean_up_sentence(sentence)
    # saco de palavras
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

def think(sentence, show_details=False):
    x = bow(sentence.lower(), words, show_details)
    if show_details:
        print ("sentence:", sentence, "\n bow:", x)
    # camada de entrada é o saco de palavras 
    l0 = x
    # multiplicação de matriz de entrada e camada oculta
    l1 = sigmoid(np.dot(l0, synapse_0))
    # camada de saída
    l2 = sigmoid(np.dot(l1, synapse_1))
    return l2

In [10]:
# E agora codificamos a função de treinamento de rede neural para criar pesos sinápticos

In [11]:
def train(X, y, hidden_neurons=10, alpha=1, epochs=50000, dropout=False, dropout_percent=0.5):

    print ("Training with %s neurons, alpha:%s, dropout:%s %s" % (hidden_neurons, str(alpha), dropout, dropout_percent if dropout else '') )
    print ("Input matrix: %sx%s    Output matrix: %sx%s" % (len(X),len(X[0]),1, len(classes)) )
    np.random.seed(1)

    last_mean_error = 1
    # inicializar aleatoriamente nossos pesos com média 0 
    synapse_0 = 2*np.random.random((len(X[0]), hidden_neurons)) - 1
    synapse_1 = 2*np.random.random((hidden_neurons, len(classes))) - 1

    prev_synapse_0_weight_update = np.zeros_like(synapse_0)
    prev_synapse_1_weight_update = np.zeros_like(synapse_1)

    synapse_0_direction_count = np.zeros_like(synapse_0)
    synapse_1_direction_count = np.zeros_like(synapse_1)
        
    for j in iter(range(epochs+1)):

       # Avançar através das camadas 0, 1 e 2
        layer_0 = X
        layer_1 = sigmoid(np.dot(layer_0, synapse_0))
                
        if(dropout):
            layer_1 *= np.random.binomial([np.ones((len(X),hidden_neurons))],1-dropout_percent)[0] * (1.0/(1-dropout_percent))

        layer_2 = sigmoid(np.dot(layer_1, synapse_1))

        # quanto perdemos o valor alvo?
        layer_2_error = y - layer_2

        if (j% 10000) == 0 and j > 5000:
            # se o erro desta iteração de 10k for maior do que a última iteração, interrompa 
            if np.mean(np.abs(layer_2_error)) < last_mean_error:
                print ("delta after "+str(j)+" iterations:" + str(np.mean(np.abs(layer_2_error))) )
                last_mean_error = np.mean(np.abs(layer_2_error))
            else:
                print ("break:", np.mean(np.abs(layer_2_error)), ">", last_mean_error )
                break
                
        # em que direção está o valor alvo?
         # em caso afirmativo, não mude muito.
        layer_2_delta = layer_2_error * sigmoid_output_to_derivative(layer_2)

        # quanto cada valor l1 contribuiu para o erro l2 (de acordo com os pesos)? 
        layer_1_error = layer_2_delta.dot(synapse_1.T)

        # em que direção está o alvo l1?
         # em caso afirmativo, não mude muito. 
        layer_1_delta = layer_1_error * sigmoid_output_to_derivative(layer_1)
        
        synapse_1_weight_update = (layer_1.T.dot(layer_2_delta))
        synapse_0_weight_update = (layer_0.T.dot(layer_1_delta))
        
        if(j > 0):
            synapse_0_direction_count += np.abs(((synapse_0_weight_update > 0)+0) - ((prev_synapse_0_weight_update > 0) + 0))
            synapse_1_direction_count += np.abs(((synapse_1_weight_update > 0)+0) - ((prev_synapse_1_weight_update > 0) + 0))        
        
        synapse_1 += alpha * synapse_1_weight_update
        synapse_0 += alpha * synapse_0_weight_update
        
        prev_synapse_0_weight_update = synapse_0_weight_update
        prev_synapse_1_weight_update = synapse_1_weight_update

    now = datetime.datetime.now()

    # persistir sinapses 
    synapse = {'synapse0': synapse_0.tolist(), 'synapse1': synapse_1.tolist(),
               'datetime': now.strftime("%Y-%m-%d %H:%M"),
               'words': words,
               'classes': classes
              }
    synapse_file = "synapses.json"

    with open(synapse_file, 'w') as outfile:
        json.dump(synapse, outfile, indent=4, sort_keys=True)
    print ("saved synapses to:", synapse_file)

In [12]:
# Agora estamos prontos para construir nosso modelo de rede neural, vamos salvá-lo como uma estrutura json para representar 
# nossos pesos sinápticos.

In [13]:
# Usamos 20 neurônios em nossa camada oculta, você pode ajustar isso facilmente. Esses parâmetros variam de acordo com 
# as dimensões e a forma de seus dados de treinamento, ajuste-os para ~ 10 ^ -3 como uma taxa de erro razoável. 

In [14]:
X = np.array(training)
y = np.array(output)

start_time = time.time()

train(X, y, hidden_neurons=20, alpha=0.1, epochs=100000, dropout=False, dropout_percent=0.2)

elapsed_time = time.time() - start_time
print ("processing time:", elapsed_time, "seconds")

Training with 20 neurons, alpha:0.1, dropout:False 
Input matrix: 12x26    Output matrix: 1x3
delta after 10000 iterations:0.006225733319071118
delta after 20000 iterations:0.004250890578071768
delta after 30000 iterations:0.0034102080724900917
delta after 40000 iterations:0.002919604826408287
delta after 50000 iterations:0.002589532257968587
delta after 60000 iterations:0.0023484431665017603
delta after 70000 iterations:0.0021626228574582566
delta after 80000 iterations:0.0020138573372354977
delta after 90000 iterations:0.001891341210744786
delta after 100000 iterations:0.001788210187496504
saved synapses to: synapses.json
processing time: 12.81472897529602 seconds


In [15]:
# Esta função classify () é tudo o que é necessário para a classificação, uma vez que os pesos das sinapses foram calculados: ~ 15
# linhas de código.
# O problema: se houver uma alteração nos dados de treinamento, nosso modelo precisará ser recalculado. Para um conjunto de 
# dados muito grande, isso pode levar um tempo não insignificante.
# Pode-se gerar a  probabilidade de uma frase pertencer a uma (ou mais) de nossas classes. Isso é super rápido porque é cálculo 
# de produto escalar em nossa função think () definida anteriormente. 

In [16]:
# limite de probabilidade
ERROR_THRESHOLD = 0.2
# carregue nossos valores de sinapse calculados
synapse_file = 'synapses.json' 
with open(synapse_file) as data_file: 
    synapse = json.load(data_file) 
    synapse_0 = np.asarray(synapse['synapse0']) 
    synapse_1 = np.asarray(synapse['synapse1'])

def classify(sentence, show_details=False):
    results = think(sentence, show_details)

    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD ] 
    results.sort(key=lambda x: x[1], reverse=True) 
    return_results =[[classes[r[0]],r[1]] for r in results]
    print ("%s \n classification: %s" % (sentence, return_results))
    return return_results

classify("sudo make me a sandwich")
classify("how are you today?")
classify("talk to you tomorrow")
classify("who are you?")
classify("make me some lunch")
classify("how was your lunch today?")
print()
classify("good day", show_details=True)

sudo make me a sandwich 
 classification: [['sandwich', 0.9983192521328343]]
how are you today? 
 classification: [['greeting', 0.9961821336974529]]
talk to you tomorrow 
 classification: [['goodbye', 0.9910402499828593]]
who are you? 
 classification: [['greeting', 0.7665674422470962], ['goodbye', 0.3010300238481574]]
make me some lunch 
 classification: [['sandwich', 0.9877818776159882]]
how was your lunch today? 
 classification: [['greeting', 0.7441825112228516], ['sandwich', 0.46171539288365376]]

found in bag: good
found in bag: day
sentence: good day 
 bow: [0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
good day 
 classification: [['greeting', 0.9968285582022675]]


[['greeting', 0.9968285582022675]]